In [21]:
import pandas as pd
import numpy as np

### Initial portfolio

In [22]:
ranks = pd.read_excel("files/ranks0.xlsx", index_col=0)
ranks = ranks.squeeze()

prices = pd.read_excel("files/prices0.xlsx", index_col=0)
prices = prices.squeeze()

equity = cash = 100000
weight = 0.2

buys = ranks<=5
buyamounts = (weight*equity/prices).astype(int)

trades = np.where(buys, buyamounts, 0)
trades = pd.Series(trades, index=ranks.index)

positions = trades
cash -= (trades*prices).sum()

### Update portfolio

In [23]:
ranks = pd.read_excel("files/ranks1.xlsx", index_col=0)
ranks = ranks.squeeze()

prices = pd.read_excel("files/prices1.xlsx", index_col=0)
prices = prices.squeeze()

equity = cash + (positions*prices).sum()
weights = (positions*prices) / equity

sellall = (ranks>10) & (positions !=0)
sellallamounts = positions
sellsome = (ranks<=10) & (weights>0.2)
sellsomeamounts = (equity*(weights-0.2)/prices).astype(int)

trades = - np.where(sellall, sellallamounts, 0)
trades -= np.where(sellsome, sellsomeamounts, 0)
trades = pd.Series(trades, index=ranks.index)

cash -= (trades*prices).sum()

tickers = ranks[positions==0].sort_values().index.to_list()

while cash >= 500:
    tick = tickers.pop(0)
    dollars = min(cash, weight*equity)
    trades[tick] = int(dollars/prices[tick])
    cash -= trades[tick]*prices[tick]


positions += trades